In [80]:
# import packages
import pandas as pd
import numpy as np
import pulp as pl

# import data
df_leads = pd.read_excel('GammaSoft data.xlsx', sheet_name='Leads').set_index('LeadID')
df_reps = pd.read_excel('GammaSoft data.xlsx', sheet_name='Reps').set_index('RepID')
df_interactions = pd.read_excel('GammaSoft data.xlsx',
                                  sheet_name='Past interactions',
                                  skiprows=1).set_index('RepID')

# revenue data
revenues = np.zeros((len(df_reps), len(df_leads)))
# Logistic regression coefficients
const    = -3.07
ext      = -0.18
past_use = 0.05
ser      = 0.42
f2f      = 0.17
for rep in df_reps.index:
    for lead in df_leads.index:
        w = ( const + ext*df_leads.loc[lead, 'EXT']
                         + past_use*df_leads.loc[lead, 'PAST_USE']
                         + ser*df_reps.loc[rep, 'SER']
                         + f2f*df_interactions.loc[rep, lead] ) 
        prob = 1/(1+np.exp(-w))
        revenues[rep,lead] = prob * df_leads.loc[lead, 'Size'] * 50

In [81]:
# created model
m = pl.LpProblem('XGen', pl.LpMaximize) # unbonded and infeasibly here won't work

# Create variables
x = []
for i in df_reps.index:
    x.append([])
    
    for j in df_leads.index:
        x[i].append( pl.LpVariable(f'x_{i}_{j}', cat='Binary') )
x = np.array(x)

# Create the constraints
for j in df_leads.index:
    m += (pl.lpSum(x[:, j]) <= 1, f'lead_{j}_constraint')
for i in df_reps.index:
    m += (pl.lpSum(x[i, :]) <= df_reps.loc[i, 'Capacity'], f'rep_{i}_constraint')
    
# Create the objective
m += pl.lpSum(revenues*x)

# check what solver is avaliable
pl.list_solvers(onlyAvailable=True)

['PULP_CBC_CMD']

In [83]:
# solve the model
pl.PULP_CBC_CMD().solve(m)

# optimal value
print(m.objective.value()) 

# variables values
for i in df_reps.index:
    print(f'Representative {i}:')
    for j in df_leads.index:
        if x[i,j].value() == 1:
            print(f'  - Lead {j}')
    print('')

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /opt/anaconda3/lib/python3.9/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/pt/ndbvf_yd23x24b9j52ckl0600000gn/T/6660c48fd5434cf09283117b5119ac8d-pulp.mps max timeMode elapsed branch printingOptions all solution /var/folders/pt/ndbvf_yd23x24b9j52ckl0600000gn/T/6660c48fd5434cf09283117b5119ac8d-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 30 COLUMNS
At line 531 RHS
At line 557 BOUNDS
At line 658 ENDATA
Problem MODEL has 25 rows, 100 columns and 200 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 110035 - 0.00 seconds
Cgl0008I 20 inequality constraints converted to equality constraints
Cgl0005I 20 SOS with 120 members
Cgl0004I processed model has 25 rows, 120 columns (120 integer (120 of which binary)) and 220 elements
Cbc0038I Initial state - 0 integers unsatisfied sum - 0
Cbc0038